In [1]:
import pandas as pd
import json
import pymongo

#set options for viewing the data
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)
pd.set_option('max_colwidth', -1)

#store csv file contents in a dataframe
inspections_data = pd.read_csv('Inspections.csv')
inventory_data = pd.read_csv('Inventroy.csv')
violations_data = pd.read_csv('Violations.csv')

#clean the inspections, extract info and save as json
inspections_data.drop_duplicates(inplace=True)
inspections_data = inspections_data.loc[inspections_data['PROGRAM STATUS'] == 'ACTIVE']
inspections_data.dropna(subset=['SCORE'], inplace=True)
inspections_data['FACILITY CITY'] = inspections_data['FACILITY CITY'].str.replace('&#160;', ' ')
inspections_data['FACILITY ADDRESS'] = inspections_data['FACILITY ADDRESS'].str.replace('&#160;', ' ') 
l = ['LOW RISK', 'MODERATE RISK', 'HIGH RISK']
inspections_data['RISK'] = inspections_data['PE DESCRIPTION'].str.extract('({})'.format('|'.join(l)),expand=False).fillna('')
inspections_data['PE DESCRIPTION'] = inspections_data['PE DESCRIPTION'].str.replace('|'.join(l), '', regex=True).str.strip()
inspections_data.to_json('Inspections.json', orient='records')

#clean the inventory data and save as json
inventory_data.drop_duplicates(inplace=True)
inventory_data['FACILITY CITY'] = inventory_data['FACILITY CITY'].str.replace('&#160;', ' ')
inventory_data['FACILITY ADDRESS'] = inventory_data['FACILITY ADDRESS'].str.replace('&#160;', ' ') 
inventory_data['OWNER CITY'] = inventory_data['OWNER CITY'].str.replace('&#160;', ' ')
inventory_data['OWNER ADDRESS'] = inventory_data['OWNER ADDRESS'].str.replace('&#160;', ' ') 
inventory_data['RISK'] = inventory_data['PE DESCRIPTION'].str.extract('({})'.format('|'.join(l)),expand=False).fillna('')
inventory_data['PE DESCRIPTION'] = inspections_data['PE DESCRIPTION'].str.replace('|'.join(l), '', regex=True).str.strip()
inventory_data.to_json('Inventory.json', orient='records')

#clean the violations data and save as json
violations_data.drop_duplicates(inplace=True)
violations_data.loc[:,'VIOLATION DESCRIPTION'] = violations_data['VIOLATION DESCRIPTION'].str.replace('[\d, "#", "."]', ' ')
violations_data.to_json('Violations.json', orient='records')

In [2]:
#connect to mongo
client = pymongo.MongoClient("mongodb://localhost:27017/")

#create database and collections
db = client['dataset']
inspections = db['inspections']
inventory = db['inventory']
violations = db['violations']
    
#function to load the json file and save as a collection
def loadJsonSave(filename, collection):
    with open(filename) as file:
        data = json.load(file)
        collection.insert_many(data)

loadJsonSave('Inspections.json', inspections)
loadJsonSave('Inventory.json', inventory)
loadJsonSave('Violations.json', violations)


In [3]:
#load from mongo and store in dataframe
inspectionsdf = pd.DataFrame(list(inspections.find({}, {'ACTIVITY DATE':1, 'FACILITY CITY':1, 'FACILITY NAME':1, 'SCORE':1, 'SERIAL NUMBER':1, '_id':0}))) 
violationsdf = pd.DataFrame(list(violations.find({},{'SERIAL NUMBER':1, 'VIOLATION DESCRIPTION':1, '_id':0})))


In [4]:
#function to calculate the mean score
def calcMean():
    return round(inspectionsdf['SCORE'].mean(), 2)

#function to calculate the median of scores
def calcMedian():
    return round(inspectionsdf['SCORE'].median(), 2)

#function to calculate the mode of scores
def calcMode():
    return inspectionsdf['SCORE'].mode()[0]


In [5]:
#function to calculate the mean score for each city
def cityMeans():
    col_name = {'mean':'MEAN'}
    return round(inspectionsdf.groupby('FACILITY CITY')['SCORE'].agg(['mean']),2).rename(columns=col_name)

#function to calculate the median of scores for each city
def cityMedians():
    col_name = {'median':'MEDIAN'}
    return round(inspectionsdf.groupby('FACILITY CITY')['SCORE'].agg(['median']),2).rename(columns=col_name)
    
#function to calculate the mode of scores for each city
def cityModes():
    col_name = {'<lambda>':'MODE'}
    return inspectionsdf.groupby('FACILITY CITY')['SCORE'].agg([lambda x: x.value_counts().index[0]]).rename(columns=col_name)


In [6]:
#create dataframe to display in GUI window
means = cityMeans()
medians = cityMedians()
modes = cityModes()
cityStats = pd.concat([means, medians, modes], axis=1)


In [7]:
#merge inspections and violations
merged = inspectionsdf.merge(violationsdf, on='SERIAL NUMBER')
merged.drop_duplicates(inplace=True)
merged['ACTIVITY DATE'] = pd.to_datetime(merged['ACTIVITY DATE'])


In [8]:
#number of establishments that have committed each type of violation 
establishments = merged['VIOLATION DESCRIPTION'].value_counts().rename_axis('VIOLATION DESCRIPTION').reset_index(name='NUMBER OF ESTABLISHMENTS')


In [9]:
#work out the most recent inspection date for each vendor
merged['LATEST DATE'] = merged.groupby('FACILITY NAME')['ACTIVITY DATE'].transform('max')


In [10]:
#drop duplicates - keep only row with the latest activity date for each vendor
latest = merged[merged['ACTIVITY DATE'] == merged['LATEST DATE']]
latest = latest.drop(['VIOLATION DESCRIPTION'], axis=1)
latest = latest.drop_duplicates()

In [11]:
#number of violations for each vendor
numberViolations = merged['FACILITY NAME'].value_counts()
numberViolations = numberViolations.rename_axis('FACILITY NAME').reset_index(name='NUMBER OF VIOLATIONS')


In [12]:
#reshape the data to determine the latest inspection score and number of violations per vendor 
violationsVsScore = numberViolations.merge(latest, on='FACILITY NAME')
violationsVsScore.drop_duplicates(subset='FACILITY NAME', keep="first", inplace=True)
violationsVsScore = violationsVsScore[['NUMBER OF VIOLATIONS', 'SCORE']]


In [13]:
from scipy import stats

#calculate pearson's correlation coefficient
pearsons = str(round(violationsVsScore['SCORE'].corr(violationsVsScore['NUMBER OF VIOLATIONS']), 3))


In [14]:
from tkinter import * 
import tkinter.scrolledtext as tkst
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure


#the main window that presents menu options
mainWindow = Tk()
mainWindow.title("Menu")

#instruction to the user
text = Label(mainWindow, text="Make a selection:")
text.grid(row=1, column=0, padx=10, pady=5)

#method to display stats options
def statsWindow():
    sWindow = Toplevel()
    sWindow.title("Statistics Menu")
    label = Label(sWindow, text='Select the statistics you would like to view')
    
    fullDataStatsButton = Button(sWindow, text="Show statistics for full dataset", width=30, command=viewFullStats)
    cityStatsButton = Button(sWindow, text='Show statistics for each facility city', width=30, command=viewCityStats)
    label.grid(row=1, column=0, padx=10, pady=5)
    fullDataStatsButton.grid(padx=55, pady=10)
    cityStatsButton.grid(padx=55, pady=10)
    
#method to view descriptive statistics for inspection score across the whole data set
def viewFullStats():
    fullStatsWindow = Toplevel()
    fullStatsWindow.title("Statistics")
    fullStatsWindow.geometry('250x150')
    label1 = Label(fullStatsWindow, text='Mean, Median and Mode for inspection score\n for the full data set:')
    labelMean = Label(fullStatsWindow, text='Mean: '+ str(calcMean()))
    labelMedian = Label(fullStatsWindow, text='Median: '+ str(calcMedian()))
    labelMode = Label(fullStatsWindow, text='Mode: '+ str(calcMode()))
    label1.grid()
    labelMean.grid(padx=10, pady=6)
    labelMedian.grid(padx=10, pady=6)
    labelMode.grid(padx=10, pady=6)
    
#method to view descriptive stats for inspection score for each facility city    
def viewCityStats():
    cityStatsWindow = Toplevel()
    cityStatsWindow.title('Facility City Statistics')
    label = Label(cityStatsWindow, text='Mean, Median and Mode for inspection score\n for each facility city:')
    editArea = tkst.ScrolledText(master = cityStatsWindow,wrap = 'word', width = 50, height = 40)
    label.grid()
    editArea.grid()
    editArea.insert('insert', cityStats)

#method for visualisation menu
def loadVisualisation():
    visualWindow = Toplevel()
    visualWindow.title("Visualisation Menu")
    label = Label(visualWindow, text='Select the visualisation you would like to view')
    establishmentViolationsButton = Button(visualWindow, text="Bar chart showing number of establishments committing violations", width=55, command=viewEstViolations)
    scoreVsViolButton = Button(visualWindow, text='Plot of inspection score and number of violations', width=55, command=scoreVsViol)
    label.grid(row=1, column=0, padx=10, pady=5)
    establishmentViolationsButton.grid(padx=55, pady=10)
    scoreVsViolButton.grid(padx=55, pady=10)
    
#method to view bar chart showing number of establishments that have committed each type of violation
def viewEstViolations():    
    newWindow = Toplevel()
    newWindow.title("Violations")
 
    fig = Figure(figsize=(15,10))
    a = fig.add_subplot(111)
    width=.5
    establishments.loc[establishments['NUMBER OF ESTABLISHMENTS']>1000].plot(kind='barh', x='VIOLATION DESCRIPTION', y='NUMBER OF ESTABLISHMENTS', legend=False, ax=a)
    
    for i in a.patches:
        a.text(i.get_width()+.1, i.get_y()+.31,str(round((i.get_width()), 2)),color='dimgrey')
        
    a.set_title('Frequency of violations across establishments where over 1000 violations recorded')
    a.set_ylabel('VIOLATION DESCRIPTION')
    a.set_xlabel('NUMBER OF ESTABLISHMENTS')
    fig.set_tight_layout(True)
    canvas = FigureCanvasTkAgg(fig, master=newWindow)
    canvas.get_tk_widget().grid(row=0, column=0)
    canvas.draw()  

#method to display scatterplot of inspection score vs number of violations
def scoreVsViol():
    window = Toplevel()
    window.title("Score Vs Violations")
    
    figure = Figure(figsize=(10,8))
    a = figure.add_subplot(111)
    a.scatter(violationsVsScore['SCORE'],violationsVsScore['NUMBER OF VIOLATIONS'])
    a.annotate("pearson's r = " + pearsons, xy=(60, 1400))
    a.set_title('Number of Violations VS Inspection Score')
    a.set_ylabel('NUMBER OF VIOLATIONS')
    a.set_xlabel('SCORE')
    canvas = FigureCanvasTkAgg(figure, master=window)
    canvas.get_tk_widget().grid(row=0, column=0)
    canvas.draw()  
    


#button options
descriptiveStatsButton = Button(mainWindow, text="Descriptive Statistics", width=20, command=statsWindow)
visualisationButton = Button(mainWindow, text= "Visualisations", width=20, command=loadVisualisation)
exitButton = Button(mainWindow, text="Exit", width=20, command=mainWindow.destroy)

#add buttons
descriptiveStatsButton.grid(padx=55, pady=10, sticky="ew")
visualisationButton.grid(padx=55, pady=10)
exitButton.grid(padx=55, pady=10)


#run the window
mainWindow.mainloop()
